In [1]:
# import statements
import numpy as np
import xarray as xr
import glob
import PolarTestingTrainingSplit_CV
import tensorflow as tf
from tensorflow import keras
import keras_tuner
from keras_tuner import BayesianOptimization
from sklearn.cross_decomposition import PLSRegression
from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)

# Get names of models in which we are testing on
path_to_data = '/home/disk/pna2/aodhan/SurfaceTrendLearning/PoChedleyEtAl2022/TASmaps/*_TrendMaps.nc'
ModelNames = [i[70:-16] for i in glob.glob(path_to_data)]

# Do train-test-split 
TrainingPredictorData, TrainingTargetData, TestingPredictorData, TestingTargetData, TestingTotalTrend = PolarTestingTrainingSplit_CV.training_testing_split(path_to_data)

2023-02-06 16:12:51.963719: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Store summaries
summaries = []

# Define the model
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(units=hp.Int('units', min_value=2, max_value=24, step=2), activation='relu', input_shape=(6,)))
    model.add(keras.layers.Dense(units=1))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-4,1e-5])), loss='mean_squared_error')
    return model

for model_idx in range(0, len(ModelNames)):
    # Define the tuner
    tuner = BayesianOptimization(build_model, objective='val_loss', max_trials=8, overwrite=True, 
    directory='/home/disk/p/aodhan/SurfaceTrendLearing/TropicalTASApplication/output',
    project_name="NN_trial_{model}".format(model=ModelNames[model_idx]), seed=0)
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

    # find specific model cross validation data
    TrainingTargetDataShape = np.shape(TrainingTargetData[model_idx])
    TestinTargetDataShape = np.shape(TestingTargetData[model_idx])
    TrainingTargetDataReshaped = np.reshape(TrainingTargetData[model_idx], (TrainingTargetDataShape[0], TrainingTargetDataShape[1]*TrainingTargetDataShape[2]))
    TestingTargetDataReshaped = np.reshape(TestingTargetData[model_idx], (TestinTargetDataShape[0], TestinTargetDataShape[1]*TestinTargetDataShape[2]))
    TrainingTargetDataReshaped = np.transpose(TrainingTargetDataReshaped[:,0])
    TestingTargetDataReshaped = np.transpose(TestingTargetDataReshaped[:,0])

    # Model Design
    pls = PLSRegression(n_components=6)
    
    # Use PLS regression to find reduced space
    pls_model = pls.fit(TrainingPredictorData[model_idx], TrainingTargetDataReshaped)
    X_train_pls = pls.transform(TrainingPredictorData[model_idx])
    X_test_pls = pls.transform(TestingPredictorData[model_idx])
    
    # Set training and testing data
    X_train =X_train_pls# np.reshape(TrainingPredictorData[model_idx], (len(TrainingPredictorData[model_idx]),72*144))
    X_test = X_test_pls#np.reshape(TestingPredictorData[model_idx], (len(TestingPredictorData[model_idx]),72*144))
    Y_train = TrainingTargetDataReshaped
    Y_test = TestingTargetDataReshaped

    # Fit the model using separate training and validation data
    history = tuner.search(X_train, Y_train, epochs=250, validation_data=(X_test, Y_test), 
                           verbose=0, callbacks=[stop_early])

    # Get best model
    best_model = tuner.get_best_models()[0]
    
    # evaluate best model
    val_loss = best_model.evaluate(X_test, Y_test, verbose=0)

    # save best hyper parameters and save them
    best_hps = tuner.get_best_hyperparameters()[0]
    summaries.append([ModelNames[model_idx], best_hps.get('units'), best_hps.get('learning_rate'), val_loss])
    
    # Print preformance and optimal hyper parameters
    print(ModelNames[model_idx])
    print('MSE:', val_loss)
    print('Optimal HPs: ', best_hps.get('units'), best_hps.get('learning_rate'))
    print('__________________________________________________')

print(summaries)

2023-02-06 16:15:31.255301: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 16:15:31.304376: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


INFO:tensorflow:Oracle triggered exit
ACCESS_ESM1_5_
MSE: 0.0007407377706840634
Optimal HPs:  12 0.0001


__________________________________________________
INFO:tensorflow:Oracle triggered exit
CESM2_
MSE: 0.00031731926719658077
Optimal HPs:  8 0.0001


__________________________________________________


In [1]:
import numpy as np
results = np.load('/home/disk/p/aodhan/SurfaceTrendLearing/TropicalTASApplication/NLPLS_1Layer_6components.npy')
print(results)

[['ACCESS_ESM1_5_' '6' '0.0001' '0.0010633140336722136']
 ['CESM2_' '16' '0.0001' '0.0003612596483435482']
 ['CNRM_CM6_1_' '14' '0.0001' '0.0001436112361261621']
 ['CanESM5_' '24' '0.0001' '0.00037265708670020103']
 ['GISS_E2_1_G_' '18' '0.0001' '0.0002152618981199339']
 ['IPSL_CM6A_LR_' '16' '0.0001' '0.0004280597495380789']
 ['MIROC_ES2L_' '14' '0.0001' '0.00012161519407527521']
 ['MIROC6_' '10' '0.0001' '0.0001678319531492889']
 ['NorCPM1_' '4' '0.0001' '0.00013684584700968117']
 ['UKESM1_0_LL_' '18' '0.0001' '0.0004776577407028526']
 ['GISS_E2_1_H_' '32' '0.0001' '0.00031077489256858826']
 ['INM_CM5_0_' '18' '0.0001' '6.888752977829427e-05']
 ['MPI_ESM1_2_HR_' '24' '0.0001' '0.00021580886095762253']
 ['MPI_ESM1_2_LR_' '22' '0.0001' '0.00010661045962478966']]


In [3]:
weighted_sum_neural = 6*40+16*50+14*29+24*40+18*12+32*16+30*14+50*10+30*4+15*18+10*32+10*18+10*24+10*22
total_models = 40+50+29+40+12+32+30+50+30+15+10+10+10
print(weighted_sum_neural/total_models)

15.094972067039107
